In [1]:
import os
import glob

In [54]:
import numpy as np
import pandas as pd

In [55]:
from PIL import Image
from sqlalchemy import create_engine

In [6]:
from ipywidgets import IntProgress
from IPython.display import display

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("seaborn-darkgrid")
plt.rcParams["figure.figsize"] = (16, 9)
plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["image.cmap"] = "Spectral"

In [8]:
eng = create_engine("sqlite:///lamost.db5")

In [9]:
key = ["teff", "logg", "feh", "planid"]

In [10]:
sql = f"SELECT {','.join(key)} FROM star"

In [11]:
data = pd.read_sql(sql, eng)
data["teff"] = np.log10(data["teff"])

In [12]:
grouped = data.groupby("planid")

```python
# incremental @ crab
prog = IntProgress(max=len(grouped)); display(prog)

plt.figure(1, figsize=(21,10), dpi=120)
plt.title("Lamost physical quantity template")
plt.xlabel("Logaritmus of $T_\mathrm{eff}$")
plt.ylabel("Logaritmus of $g$")

plt.gca().invert_xaxis();
plt.gca().invert_yaxis();
plt.xlim([3.95, 3.52]);
plt.ylim([5.15,-0.42]);

for i, plan in enumerate(grouped.groups):
    index = grouped.groups[plan]
    group = data.iloc[index]
    plt.scatter(group["teff"],
                group["logg"],
                c=group["feh"],
                s=2, alpha=0.2,
                vmin=-2.5, vmax=1);
    fln = f"pattern/{i:05d}--{plan}.png"
    if i == 0: plt.colorbar().ax.set_ylabel("Fe/H");
    plt.savefig(fln, bbox_inches="tight", pad_inches=0.5);
    prog.value = i;
    
plt.close(1);
```

```python
imag = glob.glob("pattern/*.png")
prog = IntProgress(max=len(images))
display(prog)
for i, mag in enumerate(imag):
    img = Image.open(mag)
    new = img.crop((7, 8, 1927, 1088))
    new.save(mag)
    prog.value = i
```